## Importing Dependencies

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils


## Loading of Data

Here, we are loading a combined collection of all Shakespearean sonnets. I cleaned up this file to remove the start and end credits.

In [2]:
text = (open("/content/peoms.txt").read())
text=text.lower()

## Creating character/word mappings

Now we will assign an arbitary number to each character. We are using character mapping here as our dataset is small and we have very few unique words.

In [3]:
characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}


## Data pre-processing

Here, X is our train array, and Y is our target array.

seq_length is the length of the sequence of characters that we want to consider before predicting a particular character.

In [4]:
X = []
Y = []
length = len(text)
seq_length = 100

for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

LSTMs accept input in the form of (number_of_sequences, length_of_sequence, number_of_features) which is not the current format of the arrays. Also, we need to transform the array Y into a one-hot encoded format.

In [5]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

## A deeper model

In [6]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [7]:
model.fit(X_modified, Y_modified, epochs=100, batch_size=50)




Epoch 1/100
2003/2003 [==============================] - 132s 54ms/step - loss: 2.9188
Epoch 2/100
2003/2003 [==============================] - 109s 54ms/step - loss: 2.3735
Epoch 3/100
2003/2003 [==============================] - 109s 54ms/step - loss: 2.1843
Epoch 4/100
2003/2003 [==============================] - 109s 54ms/step - loss: 2.0389
Epoch 5/100
2003/2003 [==============================] - 109s 54ms/step - loss: 1.9332
Epoch 6/100
2003/2003 [==============================] - 109s 54ms/step - loss: 1.8407
Epoch 7/100
2003/2003 [==============================] - 109s 54ms/step - loss: 1.7588
Epoch 8/100
2003/2003 [==============================] - 109s 54ms/step - loss: 1.6944
Epoch 9/100
2003/2003 [==============================] - 109s 54ms/step - loss: 1.6258
Epoch 10/100
2003/2003 [==============================] - 109s 54ms/step - loss: 1.5638
Epoch 11/100
2003/2003 [==============================] - 109s 54ms/step - loss: 1.5110
Epoch 12/100
2003/2003 [=================

In [8]:
model.save_weights('text_generator_400_0.2_400_0.2_400_0.2_100.h5')

## Generating Text

We take random row from the X array, that is an array of 100 characters. After this, we target predicting another 100 characters following X. The input is reshaped and scaled as previously and the next character with maximum probability is predicted.

seq is used to store the decoded format of the string that has been predicted till now. Next, the new string is updated, such that the first character is removed and the new predicted character is included.

In [9]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]


In [10]:
#combining text
txt=""
for char in full_string:
    txt = txt+char
txt

"\n  but as the riper should by time decease,\n  his tender heir might bear his memory:\n  but thou, control of the thing they wart,\n  who hateth thee that the world have stand thence:\n  and yet that be besper'd that heart's part,\n  still constant in a wondrous excellence;\n  therefore are feasts so shape which it doth latch:\n  of his quick objects hath the mind exes,\n  for they in thee to thy beauty show\n  that wh'touls poor inc masgents love   which thould transport me farthest from your sight.\n  b"

Our generated peom has sensible words and it also rythms. 